In [1]:
# Complete Workflow Notebook for Corrective RAG using LangGraph

# Step 1: Environment Setup
import os
from dotenv import load_dotenv, find_dotenv
from sentence_transformers import SentenceTransformer
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain.vectorstores import Chroma, FAISS
from langchain_pinecone import PineconeVectorStore
from langchain_openai import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from typing import List, Any, Tuple, Dict, Set
from langchain_core.pydantic_v1 import BaseModel, Field
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_utilization,
)
from datasets import Dataset, Features, Sequence, Value
from langgraph.graph import END, StateGraph
from typing_extensions import TypedDict
from langchain.schema import Document

# Placeholder for SemanticChunker definition or import
# from some_module import SemanticChunker

# Load environment variables from .env file
dotenv_path = find_dotenv()
if not dotenv_path:
    raise FileNotFoundError("Could not find .env file")
load_dotenv(dotenv_path)

# Azure GPT-4 model parameters
azure_api_key = os.getenv('AZURE_API_KEY')
azure_api_version = os.getenv('AZURE_API_VERSION')
azure_deployment = os.getenv('AZURE_DEPLOYMENT')
azure_endpoint = os.getenv('AZURE_ENDPOINT')

# Debugging: Print the loaded environment variables
print("AZURE_API_KEY:", azure_api_key)
print("AZURE_API_VERSION:", azure_api_version)
print("AZURE_DEPLOYMENT:", azure_deployment)
print("AZURE_ENDPOINT:", azure_endpoint)

# Check if the environment variables are loaded correctly
if not all([azure_api_key, azure_api_version, azure_deployment, azure_endpoint]):
    raise ValueError("One or more environment variables are missing.")

# Initialize the SentenceTransformer model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Define an embedding function with batching
class SentenceTransformerEmbeddings:
    def __init__(self, model, batch_size=1):
        self.model = model
        self.batch_size = batch_size

    def embed_documents(self, documents):
        embeddings = []
        for start_idx in range(0, len(documents), self.batch_size):
            batch = documents[start_idx:start_idx + self.batch_size]
            batch_embeddings = self.model.encode(batch, show_progress_bar=True).tolist()
            embeddings.extend(batch_embeddings)
        return embeddings

    def embed_query(self, query):
        return self.model.encode(query, show_progress_bar=True).tolist()

    def __call__(self, text):
        return self.model.encode(text, show_progress_bar=True).tolist()

embedding_function = SentenceTransformerEmbeddings(model, batch_size=1)

# Paths to the directories containing the PDF files
directories = {
    'chroma': r"E:\RepoFisiereMulte\FisiereTotal",
    'faiss': r"E:\RepoFisiereMulte\FisiereTotal",
    'pinecone': r"E:\RepoFisiereMulte\FisiereTotal"
}

all_texts = {key: [] for key in directories}
all_metadatas = {key: [] for key in directories}

# Process PDFs with the specified chunking method
chunking_method = 'recursive_character'

class ChunkingHelper:
    def __init__(self, file_path: str, chunking_method: str, embedding_function):
        self.file_path = file_path
        self.chunking_method = chunking_method
        self.embedding_function = embedding_function
        self.loader = PyMuPDFLoader(file_path=file_path)
        self.docs = self.loader.load()
        self.chunks = []

    def chunk_document(self):
        if self.chunking_method == 'standard_deviation':
            text_splitter = SemanticChunker(self.embedding_function, breakpoint_threshold_type='standard_deviation')
        elif self.chunking_method == 'recursive_character':
            text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        else:
            raise ValueError(f"Unknown chunking method: {self.chunking_method}")
        
        self.chunks = text_splitter.split_documents(self.docs)

    def get_chunks(self):
        if not self.chunks:
            self.chunk_document()
        return self.chunks

def process_pdfs(directory_path, all_texts, all_metadatas, chunking_method):
    for filename in os.listdir(directory_path):
        if filename.endswith(".pdf"):
            file_path = os.path.join(directory_path, filename)
            chunking_helper = ChunkingHelper(file_path, chunking_method, embedding_function)
            splits = chunking_helper.get_chunks()
            for i, split in enumerate(splits):
                split.metadata['chunk_id'] = i
                split.metadata['file'] = filename
                all_texts.append(split)
                all_metadatas.append(split.metadata)

for key, directory in directories.items():
    process_pdfs(directory, all_texts[key], all_metadatas[key], chunking_method)

index_name = "vladch"
pinecone_api_key = os.getenv('PINECONE_API_KEY')

# Check if the environment variables are loaded correctly
if not pinecone_api_key:
    raise ValueError("Pinecone API key is missing.")

# LLM for the CRAG chain
llm = AzureChatOpenAI(
    openai_api_version=azure_api_version,
    azure_deployment=azure_deployment,
    api_key=azure_api_key,
    azure_endpoint=azure_endpoint
)

# Define function to create vector stores
def create_vector_store(store_type, documents, metadatas, index_name=None):
    if store_type == 'chroma':
        return Chroma.from_texts(texts=[doc.page_content for doc in documents], embedding=embedding_function, metadatas=metadatas)
    elif store_type == 'faiss':
        return FAISS.from_texts(texts=[doc.page_content for doc in documents], embedding=embedding_function, metadatas=metadatas)
    elif store_type == 'pinecone':
        return PineconeVectorStore.from_documents(documents, index_name=index_name, embedding=embedding_function)
        

vector_stores = {key: create_vector_store(key, all_texts[key], all_metadatas[key], index_name) for key in directories}

# Create retrievers from the vector stores
retrievers = {key: store.as_retriever() for key, store in vector_stores.items()}

# Data model for grading
class GradeDocuments(BaseModel):
    binary_score: str = Field(description="Documents are relevant to the question, 'yes' or 'no'")

# LLM with function call for grading
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# Grading prompt
system_prompt = """You are a grader assessing relevance of a retrieved document to a user question. 
    If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant.
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader

# LLM for question re-writing
question_rewrite_llm = llm
question_rewrite_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """You are a question re-writer that converts an input question to a better version that is optimized for web search. Look at the input and try to reason about the underlying semantic intent / meaning."""),
        ("human", "Here is the initial question: \n\n {question} \n Formulate an improved question."),
    ]
)

question_rewriter = question_rewrite_prompt | question_rewrite_llm

# RAG Chain setup
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
llm_generate = llm
rag_chain = prompt | llm_generate | StrOutputParser()

# Graph state
class GraphState(TypedDict):
    question: str
    generation: str
    documents: List[str]
    tavily_message: str

# Define the nodes and edges for the CRAG workflow
def retrieve(state):
    print("---RETRIEVE---")
    question = state["question"]
    documents = []
    for retriever in retrievers.values():
        documents.extend(retriever.get_relevant_documents(question))
    return {"documents": documents, "question": question}

def generate(state):
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}

def grade_documents(state):
    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]
    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke({"question": question, "document": d.page_content})
        grade = score.binary_score
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
    return {"documents": filtered_docs, "question": question}

def perform_tavily_search(question):
    print("---PERFORMING TAVILY SEARCH---")
    from tavily import TavilyClient
    client = TavilyClient(api_key="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")
    tavily_results = client.search(question, search_depth="advanced")
    if isinstance(tavily_results, list):
        tavily_message = "\n".join([result["content"] for result in tavily_results])
    else:
        tavily_message = tavily_results
    print(tavily_message)
    return tavily_message

def decide_to_generate(state):
    print("---ASSESS GRADED DOCUMENTS---")
    if not state["documents"]:
        tavily_message = perform_tavily_search(state["question"])
        state["tavily_message"] = tavily_message
        state["tavily_search"] = True
        return "generate"
    else:
        state["tavily_search"] = False
        return "generate"

# Build the CRAG workflow graph
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve", retrieve)
workflow.add_node("grade_documents", grade_documents)
workflow.add_node("generate", generate)

# Build graph
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "generate": "generate",
    },
)
workflow.add_edge("generate", END)

# Compile
app = workflow.compile()

# Define the helper functions
def evaluate_answer(answer: str, question: str, context: List[Any], ground_truth: str) -> Dict[str, float]:
    context_str = format_context(context)
    context_list = context_str.split('\n\n')

    data = {
        "question": [question],
        "context": [context_str],
        "answer": [answer],
        "contexts": [context_list],
        "ground_truth": [ground_truth]
    }

    features = Features({
        "question": Value("string"),
        "context": Value("string"),
        "answer": Value("string"),
        "contexts": Sequence(Value("string")),
        "ground_truth": Value("string")
    })

    dataset = Dataset.from_dict(data, features=features)

    scores = evaluate(
        dataset,
        metrics=[context_utilization, faithfulness, answer_relevancy, context_recall],
        column_map={"question": "question", "context": "context", "answer": "answer", "contexts": "contexts", "ground_truth": "ground_truth"},
        llm=llm,
        embeddings=embedding_function
    )
    return scores

def get_aggregated_answer_and_sources(input_question: str, session_id="abc123") -> Tuple[str, Dict[str, str], List[Any], Set[Tuple[str, str]]]:
    def is_invalid_answer(answer: str) -> bool:
        return "I don't" in answer or "I'm sorry" in answer

    def get_response_and_sources(response: Dict[str, Any], all_texts: List[Any]) -> Tuple[str, List[Any], Set[Tuple[str, str]]]:
        answer = response["generation"]
        context = response["documents"]
        used_chunks = [doc for doc in all_texts if any(chunk.page_content in doc.page_content for chunk in context)]
        sources = {(chunk.metadata['file'], chunk.metadata.get('page', 'Unknown')) for chunk in used_chunks}
        return answer, context, sources

    all_responses = {}
    all_contexts = []
    all_sources = set()
    
    tavily_message = None

    for key in ['chroma', 'faiss', 'pinecone']:
        response = app.invoke({"question": input_question})
        if response.get("tavily_search"):
            tavily_message = response["tavily_message"]
            break
        if not is_invalid_answer(response["generation"]):
            answer, context, sources = get_response_and_sources(response, all_texts[key])
            all_responses[key] = answer
            all_contexts.extend(context)
            all_sources.update(sources)

    if tavily_message:
        return tavily_message, {"tavily_search": tavily_message}, [], set()

    if not all_responses:
        return "I don't know the answer.", {}, [], set()

    sentences = []
    for answer in all_responses.values():
        sentences.extend(answer.split(". "))

    sentence_embeddings = embedding_function.embed_documents(sentences)
    unique_sentences = []
    unique_embeddings = []

    for i, embedding in enumerate(sentence_embeddings):
        if not unique_embeddings:
            unique_sentences.append(sentences[i])
            unique_embeddings.append(embedding)
        else:
            similarities = cosine_similarity([embedding], unique_embeddings)
            if all(sim < 0.8 for sim in similarities[0]):
                unique_sentences.append(sentences[i])
                unique_embeddings.append(embedding)

    aggregated_answer = ". ".join(unique_sentences)

    return aggregated_answer, all_responses, all_contexts, all_sources

def format_context(context: List[Any]) -> str:
    formatted_context = "\n\n".join([
        f"File: {doc.metadata.get('file', 'Unknown')}, Page {doc.metadata.get('page', 'Unknown')}: {doc.page_content}"
        for doc in context
    ])
    return formatted_context


AZURE_API_KEY: d5c1ae66fe5a41fd82f61e0b9fe88470
AZURE_API_VERSION: 2024-02-15-preview
AZURE_DEPLOYMENT: gpt-4t
AZURE_ENDPOINT: https://itp-azopenai-ne.openai.azure.com/


C:\Users\vlad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\vlad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [2]:
# Usage Example
question = "What are the primary tasks performed by broadcast technicians during radio and television broadcasts?"
aggregated_answer, all_responses, context, sources = get_aggregated_answer_and_sources(question)

if "tavily_search" in all_responses:
    print("\nTavily Search Message:")
    print(aggregated_answer)
else:
    formatted_context = format_context(context)
    output = f"""
    Final Aggregated Answer: {aggregated_answer}

    Context used for answer:
    {formatted_context}
    """
    print(output)

    ground_truth = "Primary tasks performed by broadcast technicians during radio and television broadcasts include operating and maintaining equipment, monitoring signal strength, and ensuring quality of audio and video transmissions."

    final_scores = evaluate_answer(aggregated_answer, question, context, ground_truth)
    print("\nFinal Aggregated Answer Evaluation Scores:", final_scores)

---RETRIEVE---


C:\Users\vlad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Final Aggregated Answer Evaluation Scores: {'context_utilization': 0.8619, 'faithfulness': 0.8889, 'answer_relevancy': 0.8355, 'context_recall': 1.0000}


In [3]:
# Usage Example
question = "What were the forecasts for U.S. crude oil production in the fourth quarter of 2016?"
aggregated_answer, all_responses, context, sources = get_aggregated_answer_and_sources(question)

if "tavily_search" in all_responses:
    print("\nTavily Search Message:")
    print(aggregated_answer)
else:
    formatted_context = format_context(context)
    output = f"""
    Final Aggregated Answer: {aggregated_answer}

    Context used for answer:
    {formatted_context}
    """
    print(output)

    ground_truth = "The forecasts for U.S. crude oil production in the fourth quarter of 2016 were as follows: Current Forecast (as of December 6, 2016): 8.94 million barrels per day. Previous Forecast (as of November 8, 2016): 8.87 million barrels per day. This represents a percent change increase of 0.8% from the previous forecast to the current forecast for Q4 of 2016."

    final_scores = evaluate_answer(aggregated_answer, question, context, ground_truth)
    print("\nFinal Aggregated Answer Evaluation Scores:", final_scores)


---RETRIEVE---


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---GENERATE---
---RETRIEVE---


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Final Aggregated Answer Evaluation Scores: {'context_utilization': 0.9064, 'faithfulness': 1.0000, 'answer_relevancy': 0.9321, 'context_recall': 1.0000}


In [4]:
# Usage Example
question = "How did the number of Chapter 11 filings in Connecticut change between 1994 and 2004?"
aggregated_answer, all_responses, context, sources = get_aggregated_answer_and_sources(question)

if "tavily_search" in all_responses:
    print("\nTavily Search Message:")
    print(aggregated_answer)
else:
    formatted_context = format_context(context)
    output = f"""
    Final Aggregated Answer: {aggregated_answer}

    Context used for answer:
    {formatted_context}
    """
    print(output)

    ground_truth = "Between 1994 and 2004, the number of Chapter 11 filings in Connecticut decreased. In 1994, there were 216 filings, and by 2004 the number had declined to 84 filings. This represents a significant reduction over the ten-year period."

    final_scores = evaluate_answer(aggregated_answer, question, context, ground_truth)
    print("\nFinal Aggregated Answer Evaluation Scores:", final_scores)


---RETRIEVE---


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---GENERATE---
---RETRIEVE---


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Final Aggregated Answer Evaluation Scores: {'context_utilization': 1.0000, 'faithfulness': 1.0000, 'answer_relevancy': 1.0000, 'context_recall': 1.0000}


In [5]:
# Usage Example
question = "Who is the president of Romania?"
aggregated_answer, all_responses, context, sources = get_aggregated_answer_and_sources(question)

if "tavily_search" in all_responses:
    print("\nTavily Search Message:")
    print(aggregated_answer)
else:
    formatted_context = format_context(context)
    output = f"""
    Final Aggregated Answer: {aggregated_answer}

    Context used for answer:
    {formatted_context}
    """
    print(output)

    ground_truth = "The president of Romania is Klaus Iohannis."
    final_scores = evaluate_answer(aggregated_answer, question, context, ground_truth)
    print("\nFinal Aggregated Answer Evaluation Scores:", final_scores)


---RETRIEVE---


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---PERFORMING TAVILY SEARCH---
{'query': 'Who is the president of Romania?', 'follow_up_questions': None, 'answer': None, 'images': None, 'results': [{'title': 'Presidents Of Romania - WorldAtlas', 'url': 'https://www.worldatlas.com/articles/presidents-of-romania-since-1989.html', 'content': 'The office of President of Romania was created in 1974 when the Romanian constitution was changed via amendments to upgrade the position of President of the State Council. Since 2004, following the modification of

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Final Aggregated Answer Evaluation Scores: {'context_utilization': 0.0000, 'faithfulness': 0.6667, 'answer_relevancy': 0.0000, 'context_recall': 0.0000}


In [6]:
# Usage Example
question = "In ce data a murit Nicolae Ceausescu?"
aggregated_answer, all_responses, context, sources = get_aggregated_answer_and_sources(question)

if "tavily_search" in all_responses:
    print("\nTavily Search Message:")
    print(aggregated_answer)
else:
    formatted_context = format_context(context)
    output = f"""
    Final Aggregated Answer: {aggregated_answer}

    Context used for answer:
    {formatted_context}
    """
    print(output)

    ground_truth = "Nicolae Ceausescu a murit in data de 25 decembrie 1989"
    final_scores = evaluate_answer(aggregated_answer, question, context, ground_truth)
    print("\nFinal Aggregated Answer Evaluation Scores:", final_scores)


---RETRIEVE---


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---PERFORMING TAVILY SEARCH---
{'query': 'In ce data a murit Nicolae Ceausescu?', 'follow_up_questions': None, 'answer': None, 'images': None, 'results': [{'url': 'https://www.descopera.ro/istorie/14928957-ziua-in-care-nicolae-si-elena-ceausescu-au-fost-executati', 'title': 'Ziua în care Nicolae şi Elena Ceauşescu au fost executaţi', 'content': 'Dec 25, 2023 — Dec 25, 2023Nicolae şi Elena Ceuşescu au fost înmormântaţi la data de 30 decembrie – pe ascuns – în Cimitirul Ghencea. ... 1938 – A murit Karel 

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Final Aggregated Answer Evaluation Scores: {'context_utilization': 0.0000, 'faithfulness': 0.5000, 'answer_relevancy': 0.0000, 'context_recall': 0.0000}
